# Bridge MLS Data 🌉

In [2]:
#Import libraries
import pandas as pd
import numpy as np
import json
import requests

In [23]:
dataset_id = 'bridgemls'
server_token = '4c71c15af9f486c827e0307b79d0582a'
resource = 'property'

#r = requests.get(f'https://api.bridgedataoutput.com/api/v2/{dataset_id}/{resource}?access_token={server_token}')
r = requests.get('https://api.bridgedataoutput.com/api/v2/dataset_id/listings?access_token=access_token')

In [24]:
r

<Response [403]>

In [13]:
resource

'properties'

# RapidAPI

In [26]:
import requests

url = "https://mashvisor-api.p.rapidapi.com/rental-rates"

querystring = {"state":"CA","source":"airbnb","city":"Los Angeles","zip_code":"90291"}

headers = {
	"X-RapidAPI-Key": "552c8e0a5emsh03eb078a5d59bdap1e9190jsn571dc949c1bc",
	"X-RapidAPI-Host": "mashvisor-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'message': 'You are not subscribed to this API.'}


# Webscrape Redfin

In [ ]:
#Links
#page 1
https://www.redfin.com/city/16409/CA/Sacramento?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB')

#page 2
https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB

In [88]:
#Request
from bs4 import BeautifulSoup

r = requests.get('https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB')

In [ ]:
#Second page to scrape
#https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB

In [89]:
#Parse HTML
soup = BeautifulSoup(r.content, 'html.parser')

In [90]:
#Recieve PRICE OF HOME
prices = []
for idx in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
    try:
        prices.append(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('span', {'class': 'homecardV2Price'})[0].text)
    except:
        pass
#prices

In [91]:
property_stats = []

for idx in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
    try:
        property_stats.append((soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[0].text.split(' ')[0],
                              soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[1].text.split(' ')[0],
                              soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[2].text.split(' ')[0]))
#         print(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[0].text.split(' ')[0])
#         print(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[1].text.split(' ')[0])
#         print(soup.find_all('div', {'class': 'v2 interactive'})[idx].find_all('div', {'class': 'stats'})[2].text.split(' ')[0])
#         print()
    except:
        pass
#Bed
#bath
#sqft

In [92]:
property_names = soup.find_all('div', {'class': 'link-and-anchor'})

In [93]:
property_df = pd.DataFrame(data=list(zip(property_names, prices, property_stats)), columns=['Property_Name', 'Price', 'Stats'])

In [94]:
#Cleanse
property_df['Beds'] = property_df['Stats'].apply(lambda i: i[0])
property_df['Baths'] = property_df['Stats'].apply(lambda i: i[1])
property_df['Sqft'] = property_df['Stats'].apply(lambda i: i[2])
property_df['Property_Name'] = property_df['Property_Name'].apply(lambda i: i.text)

In [95]:
property_df.drop(columns='Stats', axis=1, inplace=True)
property_df

,Property_Name,Price,Beds,Baths,Sqft
0,"109 River Chase Cir, Sacramento, CA 95864","$728,000",3,2,"1,694"
1,"5520 Bradford Dr, Sacramento, CA 95820","$365,000",3,1,"1,139"
2,"5308 58th St, Sacramento, CA 95820","$320,000",3,1,"1,039"
3,"5622 Hillsdale Blvd Unit A, Sacramento, CA 95842","$210,000",2,1.5,933
4,"520 Santa Ana Ave, Sacramento, CA 95838","$540,000",4,2,"1,100"
5,"9982 Froom Cir, Sacramento, CA 95827","$49,950",2,1,690
6,"749 41st St, Sacramento, CA 95819","$1,649,000",5,3,"3,000"
7,"5451 70th St, Sacramento, CA 95820","$395,000",2,1,858
8,"523 18th St, Sacramento, CA 95811","$1,050,000",5,4,"2,758"
9,"3320 Sendero St, Sacramento, CA 95838","$344,900",2,1,"1,028"


# Final

#####  #Links
#page 1
https://www.redfin.com/city/16409/CA/Sacramento?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB')

#page 2
https://www.redfin.com/city/16409/CA/Sacramento/page-2?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB

In [2]:
#Request
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [27]:
#Collect All Pages of Data
prices = []
property_stats = []
property_names_ls = []
for idx in range(1,10):
    if idx < 2:
        url = 'https://www.redfin.com/city/16409/CA/Sacramento?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB'
    else:
        url = f'https://www.redfin.com/city/16409/CA/Sacramento/page-{idx}?utm_medium=ppc&utm_term=aud-923999260716:kwd-18479583&utm_content=571911945531&utm_campaign=1034079&gclid=Cj0KCQjwoeemBhCfARIsADR2QCtUnYgxvF1rdzy7Wtzu0lM2uALLwk9kDasnq185i-JZK2lPqnnGH5saAqkwEALw_wcB'
        
    #Parse HTML
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    #Recieve PRICE OF HOME
    for idx1 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
        try:
            prices.append(soup.find_all('div', {'class': 'v2 interactive'})[idx1].find_all('span', {'class': 'homecardV2Price'})[0].text)
        except:
            pass
    
    #Gather Property Stats
    for idx2 in range(len(soup.find_all('div', {'class': 'v2 interactive'}))):
        try:
            property_stats.append((soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[0].text.split(' ')[0],
                                  soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[1].text.split(' ')[0],
                                  soup.find_all('div', {'class': 'v2 interactive'})[idx2].find_all('div', {'class': 'stats'})[2].text.split(' ')[0]))
        except:
            pass
    
    property_names = soup.find_all('div', {'class': 'link-and-anchor'})
    property_names_ls = property_names_ls + [x.text for x in property_names]
    
#Convert To DF
property_df = pd.DataFrame(data=list(zip(property_names_ls, prices, property_stats)), columns=['Property_Name', 'Price', 'Stats'])

#Cleanse
property_df['Beds'] = property_df['Stats'].apply(lambda i: i[0])
property_df['Baths'] = property_df['Stats'].apply(lambda i: i[1])
property_df['Sqft'] = property_df['Stats'].apply(lambda i: i[2])

property_df.drop(columns='Stats', axis=1, inplace=True)

In [28]:
property_df

,Property_Name,Price,Beds,Baths,Sqft
0,"3829 Elm St, Sacramento, CA 95838","$279,900",3,1,918
1,"413 Tailoff Ln Unit 91B, Sacramento, CA 95818","$459,900",2,2.5,"1,116"
2,"5321 Dana, Sacramento, CA 95822","$525,000",3,2,"1,192"
3,"4900 Parker Ave, Sacramento, CA 95820","$399,000",4,2,"1,502"
4,"6735 Tortola Way, Sacramento, CA 95828","$309,000",2,2,"1,134"
...,...,...,...,...,...
345,"5360 15th Ave, Sacramento, CA 95820","$645,000",4,2,"1,630"
346,"0 Garden Hwy, Sutter, CA 95837","$257,900",—Beds,—Baths,0.61
347,"150 Del Verde Cir #98, Sacramento, CA 95833","$219,500",2,1,956
348,"3561 N St, Sacramento, CA 95816","$1,299,000",4,2,"1,260"
